In [6]:
import csv
import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
sys.path.append('../../../Lib/PythonML')

from linear import Linear
from multicouche import MLP
from random import randrange
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D

In [7]:
frequency_max = 12000
frequency_precision = 1
labels = ['guitare','saxo', 'piano']
dll_path = '../../../Lib/SupervisingML/target/debug/libmllib_rust.so'

In [8]:
def getCsvData(path):
    with open(path, newline='\n') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader, None)
        data = [] 
        count = 0
        #0,1 -> 0 2,3 -> 1 4,5 -> 2
        for row in reader:
            if len(data) == (frequency_max * 2 * frequency_precision):
                #raise ValueError('Bad data size expected ' + str(frequency_max * 2 * frequency_precision) + ' and found ' + str(len(data)) + ' for ' + path)
                break
            frequency = float(row[0])
            if frequency > frequency_max:
                break
            if int(frequency) * 2 + 1 < count:
                continue
            count += 1
            data.append([float(row[0]), float(row[1])])
       
        return data

def importData(path, label_number = len(labels)):
    data = []
    label = []
    if label_number > len(labels):
        label_number = len(labels)
    for i in range(label_number):
        directory_path = os.path.join(path, labels[i])
        for r, _, f in os.walk(directory_path):
            for file in f:
                data.append(getCsvData(os.path.join(directory_path, file)))
                label.append(i)
    return (np.array(data, dtype='float64'), np.array(label, dtype='float64'))

def shuffle(data,label):
    for i in range(len(label)):
        rand = random.randrange(i,len(label))
        label[i], label[rand] = label[rand], label[i]
        data[i], data[rand] = data[rand], data[i]
    return (data,label)

#def fit_folder(path):

In [12]:
train_data, train_label = importData('../Fourier/dataset/train', 2)
validation_data, validation_label = importData('../Fourier/dataset/validation', 2)
print("Import Finish")

train_data, train_label = shuffle(train_data, train_label)
validation_data, validation_label = shuffle(validation_data, validation_label)  
#data = np.concatenate(train_data, validation_data) 
#label = ...

Import Finish
(89, 24000, 2)


In [ ]:
train_label = to_categorical(train_label)

model = Sequential()
model.add(Dense(1, activation='linear', input_shape=train_data.shape[1:]))
model.summary()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
model.fit(train_data, train_label, epochs= 1000)

In [15]:
linear = Linear(dll_path)
linear.init_linear_model(frequency_max * 2 * frequency_precision * 2)
linear.train_linear_model_classification(train_data.flatten(), train_label.flatten(), len(train_label), 10000, 0.03)

for i in range(len(validation_data)):
    predicted_value = linear.predict_linear_model_classification(validation_data[i])
    print("predicted result : " + str(predicted_value) + " / result : " + str(validation_label[i]))

predicted result : -1.0 / result : 1.0
predicted result : 1.0 / result : 1.0
predicted result : 1.0 / result : 1.0
predicted result : -1.0 / result : -1.0
predicted result : 1.0 / result : 1.0
predicted result : 1.0 / result : -1.0
predicted result : 1.0 / result : -1.0
predicted result : 1.0 / result : 1.0
predicted result : 1.0 / result : -1.0
predicted result : -1.0 / result : 1.0
predicted result : 1.0 / result : -1.0
predicted result : -1.0 / result : 1.0
predicted result : 1.0 / result : 1.0
predicted result : 1.0 / result : -1.0
predicted result : 1.0 / result : 1.0
predicted result : -1.0 / result : -1.0
predicted result : 1.0 / result : -1.0
predicted result : 1.0 / result : -1.0
predicted result : 1.0 / result : 1.0
predicted result : 1.0 / result : 1.0
predicted result : -1.0 / result : -1.0
predicted result : 1.0 / result : 1.0
predicted result : -1.0 / result : -1.0
predicted result : -1.0 / result : -1.0


In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=data.shape[1:]),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(3)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()
history = model.fit(data, label, validation_split = 0.1, epochs=100)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
mlp = MLP(dll_path)
mlp.init_multicouche_model(np.array([train_data.shape[1] * train_data.shape[2],1], dtype='float64'))
mlp.train_multicouche_model_classification(train_data.flatten(), train_label.flatten(), train_data.shape[0], 10000.0, 0.03)

for i in range(len(validation_data)):
    predicted_value = mlp.predict_multicouche_model_classification(validation_data)[0]
    print("predicted result : " + str(predicted_value) + " / result : " + str(validation_label[i]))